# Explanations:

- выборка - тензор из картинок, таргет - вектор силы

---

Авторы используют multi output GPR, настраивая гиперпараметры $\sigma_{cov}$ и $\sigma_{err}$ (можно однозначно их выразить через гиперпараметры из того же sklearn: $l$ и $\sigma$)

GPR - непараметрический метод, суть в том, что мы делаем предположение о виде матрицы корреляции признаков для известных данных.

---

Моделирование в хотя бы немного более сложном случае буду писать на Julia

---

Гиперпараметры:

1)k: Количество элементов в массивах r_cut и p для каждого атома

2)$r_{cut}(i)_j$, i=1..k, j=1..N: векторы r_cut для j атома тоже параметр

3)$p_(i)_j$, i=1..k, j=1..N: векторы p для j атома тоже параметр

4)N_neighbours for summation for IVs

В GPyTorch есть имплементация многоразмерного регрессора: https://docs.gpytorch.ai/en/stable/examples/03_Multitask_Exact_GPs/index.html#multi-output-vector-valued-functions

---

Пока что все размерности предполагаются в системе LJ, потому что пока пытаюсь это зафитить

Исходно пока в coords.csv и forces.csv находятся для 2 частиц данные

In [32]:
import random
import os
import time

from numba import jit, njit, vectorize
import numpy as np
import scipy
from numpy.linalg import norm as norm
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

In [2]:
def set_seed(seed = 42):
    '''
    
    Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.

    '''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed()

---

# Hyperparameters:

In [39]:
class CFG:
    '''

    All hyperparameters are here

    '''
    N = 2     # число атомов
    K = 2     # можно называть это разрешением...чем число больше, тем больше размеры матрицы для атомов, фактически это число элементов в наборах p и r_cut

    p = (np.random.rand(K) + 0.1).copy()
    r_cut = (np.random.rand(K) + 0.1).copy()

    N_neig= N - 1 if N != 2 else 1

    # train_bs = 8
    # val_bs = 16
    batch_size = 8

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    f_threshold = 10    # Если сила по какой-то координате превышает это значение, то строчка исключается

---

Имеется два .csv файла:

1)
| Id(time) | 1_x | 1_y | 1_z | ... | N_z |
|------|-----|-----|-----|-----|-----|
|      |     |     |     |     |     |
|      |     |     |     |     |     |
2)
| Id(time) | f_1_x | f_1_y | f_1_z | ... | f_N_z |
|------|-----|-----|-----|-----|-----|
|      |     |     |     |     |     |
|      |     |     |     |     |     |

Одна строчка отсюда превращается в N матриц (на каждый атом) с N векторами сил

В идеале сделать БДху из двух сущностей: сила и координата, где полями будут их проекции

In [31]:
def create_df_with_coords(coords_file_path = None, forces_file_path = None):
    '''
    just makes df from .csvs with coords and forces
    '''
    coords = pd.read_csv(coords_file_path)

    forces = pd.read_csv(forces_file_path)

    if CFG.N != int(coords.columns[-1][:-1]) + 1:
        raise Exception('Constant N is not equal to amount of particles in .csv')

    return pd.merge(left=coords, right=forces, on='t').drop('t', axis='columns')

df = create_df_with_coords('coords.csv', 'forces.csv')
df

df.max()

0x          2.519317
0y          2.519801
0z        157.039283
1x        444.125151
1y          3.765265
1z          2.519838
0f_x     6300.708300
0f_y     4599.725992
0f_z      262.943956
1f_x    14945.975177
1f_y      525.511023
1f_z     1520.725800
dtype: float64

In [52]:
for numb in range(CFG.N):
    for coord in ['x', 'y', 'z']:
        

df.loc[df[abs(df['1f_z']) > 5].index]

,0x,0y,0z,1x,1y,1z,0f_x,0f_y,0f_z,1f_x,1f_y,1f_z
161,1.047038,1.473031,1.044249,0.209507,1.060259,1.461480,12.563132,6.191671,-6.258551,-12.563132,-6.191671,6.258551
162,1.043931,1.471776,1.055985,0.212613,1.061513,1.449744,17.251242,8.513640,-8.171147,-17.251242,-8.513640,8.171147
163,1.041184,1.470706,1.067817,0.215360,1.062584,1.437912,22.890348,11.312408,-10.258358,-22.890348,-11.312408,10.258358
164,1.038933,1.469890,1.079763,0.217611,1.063400,1.425966,29.352905,14.527388,-12.372807,-29.352905,-14.527388,12.372807
165,1.037348,1.469414,1.091828,0.219196,1.063876,1.413902,36.256710,17.971601,-14.272840,-36.256710,-17.971601,14.272840
...,...,...,...,...,...,...,...,...,...,...,...,...
18652,0.318435,1.602216,1.093433,0.938109,0.931074,1.412297,-27.854084,30.167584,-14.332804,27.854084,-30.167584,14.332804
18653,0.319275,1.614207,1.090613,0.937269,0.919082,1.415117,-19.877967,22.358935,-10.437791,19.877967,-22.358935,10.437791
18654,0.319586,1.626171,1.087671,0.936958,0.907118,1.418058,-13.765046,16.032146,-7.366366,13.765046,-16.032146,7.366366
18655,0.319544,1.638099,1.084654,0.937000,0.895191,1.421075,-9.220831,11.094284,-5.023973,9.220831,-11.094284,5.023973


12 индекс - 1 отн 2

$$
\vec{r_1} = \vec{r_2} + \vec{r}_{12}
$$

$$
\vec{r}_{12} = \vec{r_1} - \vec{r}_{2}
$$

In [5]:
def _get_relative_positions(row, atom_number):
    '''
    This function processes one row of csv into something that we can work with

    Returns np.array matrix that consists of relative positions vectors for passed atom_number to every other atom
    and then we can chose only closest N_neighbours in the next functions
    
    row: df.iloc[row] - typeof(row): pd.Series
    
    returns: Rel_matrix, f_vec
    '''

    s_coord = pd.Series(dtype=float)
    other_atom_numbers = [i for i in range(CFG.N) if i != atom_number]

    for other_numb in other_atom_numbers:
        index = str(atom_number) + str(other_numb)
        for axis in ['x', 'y', 'z']:
            s_coord[index + axis] = row[str(atom_number) + axis] - row[str(other_numb) + axis]

    # we need force vector only for atom_number:
    force_vec = []
    for f_axis in ['f_x', 'f_y', 'f_z']:
        force_vec.append(row[str(atom_number) + f_axis])
        

    Rel_matrix = []
    cur_vector = []

    for (i, elem) in enumerate(s_coord.values):
        if i % 3 == 0 and i != 0:
            Rel_matrix.append(cur_vector)
            cur_vector = []

        cur_vector.append(elem)
    Rel_matrix.append(cur_vector)

    return np.array(Rel_matrix), np.array(force_vec)

In [6]:
_get_relative_positions(df.iloc[0], 0)

(array([[-0.61478041,  0.82656999,  0.84915872]]),
 array([ 0.94574503, -1.2715507 , -1.30629998]))

In [7]:
from sklearn.preprocessing import normalize

In [8]:
@njit(fastmath=True)
def make_one_vec_transformed(vec, vec_norm, r_cut_i, p_i):
    '''
    vec: np.array - normalized vector
    norm: its norm
    r_cut_i: i-th component of
    '''
    return vec * np.exp(
        -np.power((vec_norm / r_cut_i), p_i)
        )

make_matrix_transformed = np.vectorize(make_one_vec_transformed)

def create_V_i(i, normalized_m, norms, r_cut=CFG.r_cut, p=CFG.p):
    '''
    normalized_m: matrix of relative distances, where rows - normalized vectors
    i: i-th component of r_cut and p, i in range 1..K (or in 0..K-1 in code)
    '''
    transf_vecs = make_matrix_transformed(normalized_m, norms[:, np.newaxis], r_cut[i], p[i])

    return np.sum(transf_vecs, axis=0)

# @njit(parallel=True)
def create_V(normalized_m, norms, K=CFG.K):
    '''
    creates V
    '''
    V = []
    for i in range(K):
        V.append(
            create_V_i(i, normalized_m, norms)
        )

    return np.array(V)

In [19]:
# @njit(
#     # parallel=True,
#     # fastmath=True
#     )
def _calculate_matrix_for_atom(relative_distances, r_cut=CFG.r_cut, p=CFG.p, N_neig=CFG.N_neig, K=CFG.K):
    '''

    relative_distances: np.array matrix of relative distance vectors

    '''

    norms = norm(relative_distances, axis=-1)
    
    # Only closest N_neig are counting:
    indexlist = np.argsort(norm(relative_distances, axis=1))
    relative_distances = relative_distances[indexlist[len(relative_distances) - N_neig:]]

    normalized_rel_distances = relative_distances / norms[:, np.newaxis]

    # print(
    #     create_V_i(0, normalized_rel_distances, norms), f'{CFG.r_cut=}, {CFG.p=}'
    # )

    V = create_V(normalized_rel_distances, norms)
    
    A = V / norm(V, axis=-1)[:, np.newaxis]

    X = V @ A.T

    return X

In [10]:
def get_matrix_for_atom(row = None, atom_number = None):
    '''

    This function will create X matrix for passed atom with
    arrays of r_cut and p of length k

    It is a wrapper for _get_relative_positions and _calculate_matrix_for_atom, so I can speed up matrix calculations
    with numba for _calculate_matrix_for_atom

    atom_number: a number of atom that we are passing
    row: one row from df_with_coords, i.e. df.iloc[index_of_row]

    '''

    # creating row of relative coordinates for concrete atom:
    relative_distances, f_vec = _get_relative_positions(row=row, atom_number=atom_number)
    X = _calculate_matrix_for_atom(relative_distances=relative_distances)
    
    return X, f_vec

# %timeit get_matrix_for_atom(row=df.iloc[0], atom_number=1)

In [11]:
from tqdm import tqdm
import gc
import torchvision.transforms as transforms

**У нас будет train и val выборки, все-таки выборку, для который известен таргет принято называть validation, на которой мы качество оцениваем, а test это все-таки выборка, для который неизвестны таргеты**

In [54]:
def clean_df(df: pd.DataFrame):
    '''

    Убирает строчки с аномально большими силами из df

    '''
    threshold = CFG.f_threshold

    return df

In [55]:
def create_tensor_dataset(coords_file_path = 'coords.csv', forces_file_path = 'forces.csv', step=1, transform=None):
    '''

    Примитивная версия датасета, просто все будет хранить в одном тензоре...

    Эта функция - wrapper на все выше написанные функции, она по переданным путям к .csv
    возвращает тензор из матриц для каждого атома в каждой строчке и тензор из векторов сил


    ИНогда есть смысл делать побольше шаг между соседними строчками, поскольку если есть почти одинаковые матрицы, то
    это по-сути линейная зависимость и модель тогда надо сильнее регулизировать

    transform: преобразование к X части датасета, в основном для нормализации нужно
    step: через сколько строчек шагать при чтении csv в датасет, чтобы уж совсем одинаковых не было

    '''

    dataset = []

    df = create_df_with_coords(coords_file_path=coords_file_path, forces_file_path=forces_file_path)#.loc[range(1000), :]

    # Сюда будет вставлена чистка df от больших сил
    df = clean_df(df)

    row_indexes = [i for i in range(0, len(df.index), step)]

    for atom_number in range(CFG.N):
        for index in tqdm(row_indexes, desc=f'Progress for atom {atom_number}'):
            row = df.iloc[index]
            x, f = get_matrix_for_atom(row=row, atom_number=atom_number)

            if transform:
                x = transform(x)
            else:
                x = transforms.ToTensor()(x)
            x = x.to(torch.float)

            dataset.append(
                (x, torch.tensor(f, dtype=torch.float))
                )
            
            # В дальнейшем для других моделей может иметь смысл хранить и возвращать тут (x, f, A), где A - соответствующая матрица для X

    gc.collect()

    return dataset

In [13]:
# Когда только начинаю работать с датасетом надо один раз на трейне посчитать std и mean, чтобы нормализовать можно было

mean = 0.14509494602680206
std = 0.09590677171945572

In [14]:
# все-таки у нас тут не картинки будут, поэтому я попробую сначала даже без нормализации, нормализовать надо 1 канал, если в терминах картинки рассуждать

transform = transforms.Compose([                                    
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std),                    
])

In [28]:
dataset = create_tensor_dataset('coords.csv', 'forces.csv', step=1, transform=transform)

Progress for atom 0:   3%|▎         | 476/19000 [00:00<00:27, 672.49it/s]/tmp/ipykernel_91737/4279244184.py:26: RuntimeWarning: invalid value encountered in true_divide
  A = V / norm(V, axis=-1)[:, np.newaxis]
/tmp/ipykernel_91737/4279244184.py:26: RuntimeWarning: invalid value encountered in true_divide
  A = V / norm(V, axis=-1)[:, np.newaxis]
/tmp/ipykernel_91737/4279244184.py:26: RuntimeWarning: invalid value encountered in true_divide
  A = V / norm(V, axis=-1)[:, np.newaxis]
/tmp/ipykernel_91737/4279244184.py:26: RuntimeWarning: invalid value encountered in true_divide
  A = V / norm(V, axis=-1)[:, np.newaxis]
/tmp/ipykernel_91737/4279244184.py:26: RuntimeWarning: invalid value encountered in true_divide
  A = V / norm(V, axis=-1)[:, np.newaxis]
/tmp/ipykernel_91737/4279244184.py:26: RuntimeWarning: invalid value encountered in true_divide
  A = V / norm(V, axis=-1)[:, np.newaxis]
/tmp/ipykernel_91737/4279244184.py:26: RuntimeWarning: invalid value encountered in true_divide
  A

KeyboardInterrupt: 

Пока никакие параметры особо не надо настраивать, поэтому и кросс валидацию не буду делать пока что, затем ее можно сделать, передавая в функцию create_dataloaders еще один параметр - фолд, на котором трейн, предварительно поделив на фолды датасет

Если просто брать в качестве трейна другие строчки из одной генерации, то можно не отследить переобучения, стоит пробовать тестить на датасете, который отдельно сгенерирован с таким же числом частиц, который модель еще вообще не видела

In [23]:
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(dataset, test_size=0.33, random_state=42)

train_data[62]

(tensor([[[ 0.1457,  0.1457],
          [-1.4108, -1.4108]]]),
 tensor([-0.0040,  0.0084,  0.0057]))

**Код для выяснения mean и std у трейновой выборки: (по ненормализованному датасету делается)** 

In [24]:
def get_mean_and_std_for_train_X(train_data):
    train_X = torch.cat([row[0] for row in train_data])

    print(
        f'mean = {torch.mean(train_X)}, std = {torch.std(train_X)}'
    )

get_mean_and_std_for_train_X(train_data=train_data) # тупо проверка

mean = nan, std = nan


## Когда молекул уже будет много как хранить данные:

In [18]:
# Эта клетка нужна будет, когда молекул будет много (N > 100, K порядка 100)

def create_df_with_paths(df_coords: pd.DataFrame, first_folder = 'Atom_matrices'):
    '''

    Пока эта функция не нужна, но в будущем за счет нее как раз будет работать PathBasedDataset

    gets df, returns df with paths to torch matrices for each atom for different times,
    basically this function will call get_matrix_for_atom a lot of times

    output: pd.DataFrame that orignated from this:
    
    | Index | 1_atom_X_path                     | ... | N_atom_X_path                     |
    |-------|-----------------------------------|-----|-----------------------------------|
    | 1     | ./atom_matrices/index1/atom1.tb   |     | ./atom_matrices/index1/atomN.tb   |
    | ...   |                                   |     |                                   |
    | 30k   | ./atom_matrices/index30k/atom1.tb |     | ./atom_matrices/index30k/atomN.tb |
    
    but eventually will look like this:

    | Index   | atom_X_path                       |
    |---------|-----------------------------------|
    | 1       | ./atom_matrices/index1/atom1.tb   |
    | ...     | ...                               |
    | 30k * N | ./atom_matrices/index30k/atomN.tb |

    '''
    row_numbers = df_coords.index

    df_paths = pd.DataFrame(
        {
            'path': []
        }
    )

    pass

class PathBasedDataset(torch.utils.data.Dataset):
    '''

    Это будет класс датасета из торча для большого числа молекул, если молекул будет очень много, то надо будет уже хранить все матрицы X не в оперативной памяти

    При создании экземпляра будет передаваться pd.Dataframe, который
    состоит из трех колонок - проекций вектора силы и еще одной колонки - путь к файлу, где лежит как-то заэнкоженная
    матрица для данного атома, и так для каждого атома (я проверил, что запись и чтение при помощи torch.save и torch.load для тензоров очень быстрое)

    '''
    def __init__(self, df, transforms=None, mode='train'):
        self.df = df    # it will be dataframe with coordinates and forces of all atoms
        self.mode = mode
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        x = 1   # it will be a matrix KxK for each atom
        y = 1   # it will be a force vector with shape: (3)

        if self.mode == 'test':
            return x
        return x, y

In [19]:
from torch.utils.data import DataLoader

In [20]:
def create_dataloaders(train_dataset, val_dataset, train_bs=CFG.batch_size, val_bs=CFG.batch_size, fold=None):
    '''

    Returns train_loader, val_loader

    fold: will be used in cross validation, when I will implement it

    '''
    
    train_loader = DataLoader(dataset=train_dataset, batch_size=train_bs, shuffle=True)

    val_loader = DataLoader(dataset=val_dataset, batch_size=val_bs, shuffle=False)

    return train_loader, val_loader

In [21]:
train_loader, val_loader = create_dataloaders(train_data, val_data)

---

# Обучение:

Я попробую оба варианта:
1) Многомерный аутпут
2) Для каждой компоненты свой одномерный аутпут

надо помнить, что сначала на двух частицах!

## 1)Многомерный аутпут:

In [22]:
import torch.nn.functional as F

Вообще постоянный множитель - это не особо важно, но просто при оценке качества модели возникнут определенные трудности

In [23]:
class MultuOutputMSELoss(nn.MSELoss):
    '''

    Custom loss that calculates average over batch loss for multidim MSE - sum of MSE for components

    Example:
    |
    |    Loss = MultuOutputMSELoss()
    |
    |    a = torch.ones((8, 3))      # it is batch of 8 vectors of size 3
    |    b = torch.zeros((8, 3))
    |
    |    Loss(a, b, batch_size=8) -> 3

    '''

    def forward(self, input, target, batch_size=CFG.batch_size):
        '''
        оно при reduction='mean' делит на произведение всех размерностей
        '''
        return F.mse_loss(input, target, reduction='sum') / batch_size   # или эквивалентно делать reduction='mean' и умножать на input.size()[-1] - length of output

In [24]:
L = MultuOutputMSELoss()

a = torch.tensor([
    [1, 2, 3],
    [1, 2, 3],
    [1, 2, 3],
    [1, 2, 3],
    [1, 2, 3]
])
b = torch.tensor([
    [1, 0, 1],
    [1, 0, 0],
    [1, 0, 0],
    [1, 0, 1],
    [1, 0, 1]
], dtype=torch.double)

L(a, b, 5)

tensor(10., dtype=torch.float64)

In [25]:
class SingleNet(nn.Module):
    def __init__(self, output_size):
        super().__init__()

        # Когда будет много частиц, здесь можно свертки и пуллинги запихать, в FC сетку уже надо подавать распрямленную матрицу
        # self.conv1 = ...

        self.fully_connected = nn.Sequential(
            nn.Linear(CFG.K * CFG.K, 1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.BatchNorm1d(1024),

            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.BatchNorm1d(512),
            nn.Linear(512, output_size),
        )

    def forward(self, x):
        # x - is batch of matrices KxK

        # Здесь происходят какие-то там свертки, пуллинги и тп...

        x = x.view(x.size(0), -1)
        x = self.fully_connected(x)

        return x

В качестве метрики буду использовать сумму MSE по компонентам, лоссы попробую разные

In [26]:
def fit_epoch(model, train_loader, criterion, optimizer, scheduler, scaler=None):
    '''

    Функция обучения по всем батчам 1 раз (1 эпоха)

    scaler: gradient scaler from torch.amp, попозже добавлю обучение с ним

    '''
    running_loss = 0.0
    running_MSE = 0
    processed_data = 0
  
    for inputs, labels in train_loader:
        
        inputs = inputs.to(CFG.device)
        labels = labels.to(CFG.device)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_MSE += F.mse_loss(input=outputs, target=labels, reduction='sum').item()
        processed_data += inputs.size(0)

    scheduler.step()
              
    train_loss = running_loss / processed_data
    train_MSE = running_MSE / processed_data
    
    return train_loss, train_MSE

In [27]:
def eval_epoch(model, val_loader, criterion):

    model.eval()
    
    running_loss = 0.0
    running_MSE = 0
    processed_size = 0

    for inputs, labels in val_loader:
        inputs = inputs.to(CFG.device)
        labels = labels.to(CFG.device)

        with torch.no_grad():
            outputs = model(inputs)
            loss = criterion(outputs, labels)

        running_loss += loss.item() * inputs.size(0)
        running_MSE += F.mse_loss(input=outputs, target=labels, reduction='sum')
        processed_size += inputs.size(0)

    val_loss = running_loss / processed_size
    val_MSE = running_MSE.double() / processed_size
    
    return val_loss, val_MSE

In [28]:
def train(train_loader, val_loader, model, optimizer, scheduler, epochs, scaler=None, criterion=MultuOutputMSELoss()):
    '''

    Полный цикл обучения
    
    '''

    history = []
    log_template = "\nEpoch {ep:03d} train_loss: {t_loss:0.4f} \
    val_loss {v_loss:0.4f} train_acc {t_mse:0.4f} val_acc {v_mse:0.4f}"

    with tqdm(desc="epoch", total=epochs) as pbar_outer:

        for epoch in range(epochs):
            train_loss, train_MSE = fit_epoch(model, train_loader, criterion, optimizer, scheduler)
            print("loss", train_loss)

            val_loss, val_MSE = eval_epoch(model, val_loader, criterion)
            if epoch != 0:
                if history[-1][-1] < val_MSE:
                    torch.save(model.state_dict(), './model.pth')     # сохраняем модель напрямую в гугл диск 
            
            history.append((train_loss, train_MSE, val_loss, val_MSE))
            
            pbar_outer.update(1)
            tqdm.write(log_template.format(ep=epoch + 1, t_loss=train_loss,\
                                           v_loss=val_loss, t_mse=train_MSE, v_mse=val_MSE))
            
            
    return history

In [29]:
import torch.optim as optim
from torch.optim import lr_scheduler

In [30]:
model = SingleNet(output_size=3).to(CFG.device)

optimizer = optim.Adam(model.parameters(), lr=7e-3, betas=(0.9, 0.999), weight_decay=0.01)

# scheduler.step нужно первый раз делать обязательно после optimizer.step, потому что иначе мы просто пропустим первый шаг scheduler
exp_scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

Попытка для двух частиц:

In [31]:
history = train(
    train_loader=train_loader, val_loader=val_loader, model=model, optimizer=optimizer,
    scheduler=exp_scheduler, epochs=10
)

epoch:   0%|          | 0/10 [00:00<?, ?it/s]

loss 154.77891783723658


epoch:  10%|█         | 1/10 [00:04<00:39,  4.38s/it]


Epoch 001 train_loss: 154.7789     val_loss 139.9067 train_acc 154.7793 val_acc 140.0669
loss 185.592015986937


epoch:  20%|██        | 2/10 [00:08<00:32,  4.03s/it]

...


Epoch 008 train_loss: 149.4645     val_loss 136.3212 train_acc 149.9463 val_acc 136.4843
loss 149.9332806679178


epoch:  90%|█████████ | 9/10 [00:33<00:03,  3.68s/it]


Epoch 009 train_loss: 149.9333     val_loss 136.3332 train_acc 149.9336 val_acc 136.4969
loss 149.86301196442753


epoch: 100%|██████████| 10/10 [00:37<00:00,  3.75s/it]


Epoch 010 train_loss: 149.8630     val_loss 136.2958 train_acc 149.8633 val_acc 136.4583
